## Parameter Recovery 
### Formatting for matlab analysis 


In [1]:
# preamble and imports 
import arviz as az
import os, glob
from cmdstanpy import from_csv 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.io import savemat

In [2]:
# loading in the parameter recivery fits 
# post-processing into matlab structs here 
# model_names = ['AlphaSMP', 'AlphaSMEP','BayesSMP', 'BayesSMEP'] #removed delta models

model_names = ['BayesSMEP']

groups      = ['HC', 'PreTreat', 'PostTreat', 'PostTreat_Liv', 'PostTreat_Dun']
nSubjectsGroups = np.array([32,37,37,25,12])

params = {
    'AlphaSM':   ['alpha','beta'],
    'AlphaSME':  ['alpha','beta','phi'],
    'AlphaSMP':  ['alpha','beta','persev'],
    'AlphaSMEP': ['alpha','beta','phi','persev'],
    'BayesSM':   ['beta'],
    'BayesSME':  ['beta','phi'],
    'BayesSMP':  ['beta','persev'],
    'BayesSMEP': ['beta','phi','persev'],
}

param_list = ["alpha","beta","phi","persev"]  # columns

colors = sns.color_palette("deep", 5)

palette = dict(zip(groups, sns.color_palette('deep', n_colors=len(groups)).as_hex()))

In [3]:
def load_param_recovery_as_az(root_dir, groups, models):
    idatas = {}
    for group in groups:
        idatas[group] = {}
        for model in models:
            idatas[group][model] = {}

            model_dir = os.path.join(root_dir, group, model)
            subject_dirs = sorted(glob.glob(os.path.join(model_dir, 'subject_*')))

       
            for subject_dir in subject_dirs:
               
                csvs = sorted(glob.glob(os.path.join(subject_dir, '*.csv')))
      
                fit = from_csv(csvs)

                idatas[group][model][os.path.basename(subject_dir)] = az.from_cmdstanpy(fit)

    return idatas

In [4]:
path_to_fits = 'fits'
idatas_groups = load_param_recovery_as_az(path_to_fits, groups, model_names)

In [6]:
# checking fit keys 
posterior = idatas_groups['HC']['BayesSMEP']['subject_01'].posterior
for param in params['BayesSMEP']:
    print(param)


for v in posterior.data_vars:
    print(v, "dims:", posterior[v].dims, "shape:", posterior[v].shape)


beta
phi
persev
beta dims: ('chain', 'draw', 'beta_dim_0') shape: (4, 2500, 15)
phi dims: ('chain', 'draw', 'phi_dim_0') shape: (4, 2500, 15)
persev dims: ('chain', 'draw', 'persev_dim_0') shape: (4, 2500, 15)


In [7]:
# And load the Parameters to be Recovered 
nK = 5
nSubSims = 3
nDraws = 10000  # fixed: 4 chains * 2500 draws

pr_fits= {}

for group in groups:
    pr_fits[group] = {}
    for model in model_names:
        pr_fits[group][model] = {}
        
        subjects_dict = idatas_groups[group][model] 
        for subject in subjects_dict:
           
            posterior = subjects_dict[subject].posterior
            subj_params = {}

            for p in params[model]:
          
                x = posterior[p].stack(sample=('chain', 'draw')).values  # (10000, 15)

                x = x.reshape(nDraws, nK, nSubSims).transpose(1, 2, 0)
                subj_params[p] = x

            pr_fits[group][model][subject] = subj_params
  
savemat('parameter_recovery_draws.mat', pr_fits, do_compression=True, long_field_names=True)